In [1]:
import numpy as np
from tabulate import tabulate

In [2]:
criteria = np.loadtxt('task_matrix.txt', delimiter=' ', dtype=int)
criteria

array([[10,  3, 10, 10,  4,  3,  4,  1,  8,  3,  8,  4],
       [ 8,  3,  8,  7,  2,  3,  4,  1,  8,  3,  8,  4],
       [10,  8, 10, 10,  4,  9,  4,  3,  8,  4,  8,  7],
       [10,  8, 10, 10,  5,  9,  8,  4,  8,  5,  8,  7],
       [ 5,  3,  4,  6,  2,  3,  4,  1,  1,  3,  1,  4],
       [ 3,  2,  4,  5,  2,  3,  4,  1,  1,  2,  1,  4],
       [10,  8, 10, 10,  7,  9,  4,  8,  8,  5,  8,  7],
       [10,  8, 10, 10,  7,  9,  4,  8,  8,  7,  8,  7],
       [10,  8, 10, 10,  7,  9,  9,  8, 10, 10,  8,  7],
       [ 1,  4,  2,  5,  7,  8,  9,  8,  1,  1,  2,  7],
       [ 1,  2,  2,  5,  2,  3,  4,  1,  1,  1,  1,  4],
       [ 1,  2,  2,  1,  2,  3,  4,  1,  1,  1,  1,  4],
       [10,  8, 10, 10,  7,  9,  9,  8, 10, 10,  8, 10],
       [10,  9, 10, 10,  7,  9,  9,  8, 10, 10,  8, 10],
       [ 2,  7, 10,  1,  7,  1,  4,  8,  5,  7,  6,  7],
       [ 2,  7, 10,  1,  7,  3,  4,  8,  5,  9,  6,  8],
       [10,  9, 10, 10, 10,  4,  4,  8,  8,  9,  8,  8],
       [10,  9, 10, 10, 10,  4,

#### Формуємо множину веторів сигма.

In [3]:
def make_delta_vector(criteria, index_row): #утворення множини векторів дельта, щоб потім використати для знаходження сигм
    deltas = []
    for i in range(criteria.shape[0]):
        deltas.append(criteria[index_row]-criteria[i])
    deltas = np.array(deltas)
    return deltas
    
def sigma_vectors(criteria):
    sigma_vect = []
    
    for i in range(criteria.shape[0]):
        delta = make_delta_vector(criteria, i)
        
        delta_positive = np.where(delta > 0, 1, 0)
        delta_negative = np.where(delta < 0, -1, 0)

        sigma = delta_positive+delta_negative
        sigma_vect.append(sigma)
        
    sigma_vect = np.array(sigma_vect)
    return sigma_vect

In [4]:
sigma_vect = sigma_vectors(criteria)
sigma_vect

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  1, ...,  0,  0,  0],
        [ 0, -1,  0, ..., -1,  0, -1],
        ...,
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 1, -1,  1, ..., -1,  1,  1]],

       [[-1,  0, -1, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [-1, -1, -1, ..., -1,  0, -1],
        ...,
        [-1, -1, -1, ..., -1,  0, -1],
        [-1, -1, -1, ..., -1,  0, -1],
        [ 1, -1,  1, ..., -1,  1,  1]],

       [[ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  1,  1, ...,  1,  0,  1],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 1, -1,  1, ...,  0,  1,  1]],

       ...,

       [[ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  1,  1, ...,  1,  0,  1],
        [ 0,  1,  0, ...,  1,  0,  1],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0, -1,  0, ...,  0,  0, -1],
        [ 1, -1,  1, ...,  1,  1

#### Реалізація функцій для для оптимізації за домінуванням і блокуванням

In [5]:
def I(R): #формуємо симетричну частину
    return (R == R.T)*R

def P(R):#формуємо асиметричну частину
    return R-I(R)

def N(R):#формуємо відношення непорівнюваності
    return (R == R.T)-I(R)

def check_asymmetry(R):#перевірка на асиметричність. True - якщо асиметричне
    return not I(R).any()

#Оптимізація за домінуванням
def dom_opt(R, m):
    opt_res = {
        'l_p': [],
        'l_r': [],
        'strong_l_r': []}
    
    if check_asymmetry(R):
        opt_res['l_p'] = largest_p(R, m)
    else:
        opt_res['l_r'], opt_res['strong_l_r'] = largest_r(R, m)
    return opt_res

def largest_p(R, m):#найбільші по P
    for row, i in zip(R, range(m)):
        if row.sum()==m-1 and row[i]==0:
            return [i+1] #бо одночасно 2 таких елементи існувати не може
    return []

def largest_r(R, m):#найбільші та строго найбільші по R 
    lr, strong_lr = [], []
    for row, i in zip(R, range(m)):
        if row.sum()==m:
            lr.append(i+1)
            if R.T[i].sum()==1:
                strong_lr.append(i+1)
    return lr, strong_lr
            
#Оптимізація за блокуванням
def block_opt(R, m):
    opt_res = {
        'm_p': [],
        'm_r': [],
        'strong_m_r': []}
        
    if check_asymmetry(R):
        opt_res['m_p'] = max_p(R, m)
    else:
        opt_res['m_r'], opt_res['strong_m_r'] = max_r(R, m)
    return opt_res

def max_p(R, m):#максимальні по P
    mp = []
    for row, i in zip(R.T, range(m)):
        if not row.any():
            mp.append(i+1)
    return mp

def max_r(R, m):#максимальні та строго максимальні по R
    mr, strong_mr = [], []
    for row, i in zip(P(R).T, range(m)):
        if not row.any():
            mr.append(i+1)
            if R.T[i].sum()==0 or (R.T[i].sum()==1 and R.T[i][i] == 1):
                strong_mr.append(i+1)
    return mr, strong_mr

def optimization(R):
    m = R.shape[0]
    headers = ["Множини альтернатив","Оптимальний розв'язок"]
    
    solution = dom_opt(R, m)
    if len(solution['l_p']) or len(solution['l_r']) or len(solution['strong_l_r']):
        if len(solution['l_p']) == 0:
            del solution['l_p']
        else:
            del solution['l_r']
            del solution['strong_l_r']
        print(tabulate(solution.items(), headers = headers))
    else:
        solution = block_opt(R, m)
        if len(solution['m_p']) == 0:
            del solution['m_p']
        else:
            del solution['m_r']
            del solution['strong_m_r']
        print(tabulate(solution.items(), headers = headers))

#### Відношення Парето

In [6]:
def pareto(sigma_vect):
    m = sigma_vect.shape[0] #кількість x
    solution = np.zeros((m, m), dtype=int)
    
    for row in range(m):
        for col in range(m):
            if (sigma_vect[row][col] >= 0).all():
                solution[row][col] = 1
    
    return solution

In [7]:
pareto_mtrx = pareto(sigma_vect)
print("Відношення Парето")
pareto_mtrx

Відношення Парето


array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
print("ВІДНОШЕННЯ ПАРЕТО\n____________________________________")
optimization(pareto_mtrx)

ВІДНОШЕННЯ ПАРЕТО
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
m_r                    [14, 19]
strong_m_r             [14, 19]


#### Мажоритарне відношення

In [9]:
def majority(sigma_vect):
    m = sigma_vect.shape[0] #кількість x
    sigma_sum = np.array([[sigma_vect[i][j].sum() for j in range(m)] for i in range(m)])
    Pm = np.where(sigma_sum > 0, 1, 0)
    return Pm

In [10]:
majority_mtrx = majority(sigma_vect)
majority_mtrx

array([[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 0,

In [11]:
print("МАЖОРИТАРНЕ ВІДНОШЕННЯ\n____________________________________")
optimization(majority_mtrx)

МАЖОРИТАРНЕ ВІДНОШЕННЯ
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
l_p                    [14]


#### Відношення лексикографії

In [12]:
# Важливості критеріїв k4>k2>k10>k1>k3>k9>k12>k5>k11>k7>k6>k8
def lexicographic(sigma_vect):
    Pl=np.full(400, 2).reshape(20,20)
    K = [3,1,9,0,2,8,11,4,10,6,5,7]
    
    for k in K:
        for row, i_row in zip(sigma_vect, range(sigma_vect.shape[0])):
            for col, i_col in zip(row, range(sigma_vect.shape[1])):
                if Pl[i_row][i_col] == 2:
                    if col[k]==1:
                        Pl[i_row][i_col]=1
                    elif col[k]== -1:
                        Pl[i_row][i_col]=0
    Pl = np.where(Pl==2, 0, Pl)
    return Pl

In [13]:
lexicographic_mtrx = lexicographic(sigma_vect)
lexicographic_mtrx

array([[0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
print("ВІДНОШЕННЯ ЛЕКСИКОГРАФІЇ\n____________________________________")
optimization(lexicographic_mtrx)

ВІДНОШЕННЯ ЛЕКСИКОГРАФІЇ
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
l_p                    [19]


#### Відношення Березовського

In [68]:
# Відношення квазіпорядку на мн-ні критеріїв (класи впорядковані за зростанням важливості):
#{k8,k9,k12} < {k1,k3,k10,k11} < {k2,k4,k5,k6,k7}

def beresovskiy(criteria):
    K1 = [7,8,11]
    K2 = [0,2,9,10]
    K3 = [1,3,4,5,6]
    
    Ib_2, Pb_2, Nb_2, solution = [[0]*20 for i in range(20)],[[0]*20 for i in range(20)],[[0]*20 for i in range(20)],[[0]*20 for i in range(20)]
    
    #формуємо матриці векторів сигм для кожного з класів
    sigma_K1 = sigma_vectors(criteria[:, K1])
    sigma_K2 = sigma_vectors(criteria[:, K2])
    sigma_K3 = sigma_vectors(criteria[:, K3])
    
    #формуємо відношення Парето для кожного з класів
    pareto_K1 = pareto(sigma_K1)
    pareto_K2 = pareto(sigma_K2)
    pareto_K3 = pareto(sigma_K3)
    
    #виділяємо симетричну частину для кожного з класів
    I0_K1 = I(pareto_K1)
    I0_K2 = I(pareto_K2)
    I0_K3 = I(pareto_K3)
    
    #виділяємо асиметричну частину для кожного з класів
    P0_K1 = P(pareto_K1)
    P0_K2 = P(pareto_K2)
    P0_K3 = P(pareto_K3)
    
    #виділяємо непорівнювану частину для кожного з класів
    N0_K1 = N(pareto_K1)
    N0_K2 = N(pareto_K2)
    N0_K3 = N(pareto_K3)
    
    #1 ітерація
    Pb_1 = P0_K1
    Ib_1 = I0_K1
    Nb_1 = N0_K1
    
    #2 ітерація
    #Pb2
    Pb_2_1 = (P0_K2==P0_K1)*P0_K2
    Pb_2_2 = (P0_K2==I0_K1)*P0_K2
    Pb_2_3 = (P0_K2==N0_K1)*P0_K2
    Pb_2_4 = (I0_K2==P0_K1)*I0_K1
    Pb_2 = Pb_2_1+Pb_2_2+Pb_2_3+Pb_2_4
    Pb_2 = np.where(Pb_2==0, 0, 1)

    #Ib2
    Ib_2 = (I0_K2==I0_K1)*I0_K2
    
    #Nb2
    Nb_2 = N(Pb_2+Ib_2)
    
    # 3 ітерація
    Pb_3_1 = (P0_K3==Pb_2)*Pb_2
    Pb_3_2 = (P0_K3==Ib_2)*Ib_2
    Pb_3_3 = (P0_K3==Nb_2)*Nb_2
    Pb_3_4 = (I0_K3==Pb_2)*Pb_2
    Pb_3 = Pb_3_1+Pb_3_2+Pb_3_3+Pb_3_4
    Pb_3 = np.where(Pb_3==0, 0, 1)
    
    return Pb_3

In [69]:
beresovskiy_mtrx = beresovskiy(sigma_vect)
beresovskiy_mtrx

array([[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [70]:
print("ВІДНОШЕННЯ БЕРЕЗОВСЬКОГО\n____________________________________")
optimization(beresovskiy_mtrx)

ВІДНОШЕННЯ БЕРЕЗОВСЬКОГО
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
m_p                    [14, 18, 19]


#### Відношення Подиновського

In [71]:
def sort_criteria(criteria):
    sort_crit = np.array(criteria)
    for row, i in zip(sort_crit, range(criteria.shape[0])):
        row[::-1].sort()
        sort_crit[i] = row
    return sort_crit

def podynovskiy(criteria):
    sort_crit = sort_criteria(criteria)
    sigma_vect = sigma_vectors(sort_crit)
    pareto_mtrx = pareto(sigma_vect)
    return pareto_mtrx

In [72]:
podynovskiy_mtrx = podynovskiy(criteria)
podynovskiy_mtrx

array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0,

In [73]:
print("ВІДНОШЕННЯ ПОДИНОВСЬКОГО\n____________________________________")
optimization(podynovskiy_mtrx)

ВІДНОШЕННЯ ПОДИНОВСЬКОГО
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
l_r                    [14]
strong_l_r             [14]


#### Запис відношень у файл

In [74]:
with open("Var09-МозольоваДарина.txt", "w") as result:
    for matrix, i in zip([pareto_mtrx, majority_mtrx, lexicographic_mtrx, beresovskiy_mtrx, podynovskiy_mtrx], list(range(1, 6))):
        result.write(" {} \n".format(i))
        for row in range(20):
            for col in range(20):
                result.write(" {} ".format(matrix[row][col]))
            result.write("\n")